In [1]:
import glob
from PIL import Image
import numpy as np
import pandas as pd

In [2]:
no_smile = glob.glob("../input/smiling-or-not-face-data/non_smile/*.jpg")
smile = glob.glob("../input/smiling-or-not-face-data/smile/*.jpg")
test = glob.glob("../input/smiling-or-not-face-data/test/*.jpg")

In [3]:
X=[]
y=[]
test_data = []
for i in no_smile:
    X_i = Image.open(i)
    X_i = np.array(X_i.resize((32,32)))/255
    X.append(X_i)
    y.append("no smile")
    
for i in smile:
    X_i = Image.open(i)
    X_i = np.array(X_i.resize((32,32)))/255
    X.append(X_i)
    y.append("smile")
    
for i in test:
    X_i = Image.open(i)
    X_i = np.array(X_i.resize((32,32)))/255
    test_data.append(X_i)

In [4]:
X = np.array(X)
y = np.array(y)
test = np.array(test)
from sklearn.utils import shuffle
X,y = shuffle(X,y,random_state=0)

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [6]:
print("{} \n{}".format(X.shape,y.shape))

(1203, 32, 32, 3) 
(1203,)


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)
print("{} \n{}".format(X_train.shape,X_test.shape))
print("{} \n{}".format(y_train.shape,y_test.shape))

(962, 32, 32, 3) 
(241, 32, 32, 3)
(962,) 
(241,)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Flatten,MaxPooling2D,Dropout,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
callback = EarlyStopping(patience=60)

In [10]:
model = Sequential([Conv2D(128,(3,3),activation="relu",input_shape= X_train.shape[1:]),
                    MaxPooling2D((2,2)),
                    Dropout(0.3),
                    BatchNormalization(),
                    Conv2D(64,(3,3),activation="relu"),
                    MaxPooling2D((2,2)),
                    Conv2D(64,(3,3),activation="relu"),
                    Flatten(),
                    Dense(128,activation="relu"),
                    Dropout(0.3),
                    BatchNormalization(),
                    Dense(128,activation="relu"),
                    Dense(1,activation="sigmoid")])
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 128)       3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 128)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 15, 128)       512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          3

In [11]:
train_hist = model.fit(X_train,y_train,epochs=80,batch_size=8,callbacks=[callback],validation_split=0.1)

Epoch 1/80
109/109 [==============================] - 4s 26ms/step - loss: 0.6286 - accuracy: 0.6162 - val_loss: 0.6593 - val_accuracy: 0.5052
Epoch 2/80
109/109 [==============================] - 3s 25ms/step - loss: 0.3973 - accuracy: 0.8301 - val_loss: 0.5439 - val_accuracy: 0.7732
Epoch 3/80
109/109 [==============================] - 4s 35ms/step - loss: 0.4197 - accuracy: 0.8150 - val_loss: 0.5622 - val_accuracy: 0.6907
Epoch 4/80
109/109 [==============================] - 2s 23ms/step - loss: 0.3538 - accuracy: 0.8636 - val_loss: 0.3940 - val_accuracy: 0.8557
Epoch 5/80
109/109 [==============================] - 3s 24ms/step - loss: 0.3220 - accuracy: 0.8694 - val_loss: 0.3318 - val_accuracy: 0.9072
Epoch 6/80
109/109 [==============================] - 3s 23ms/step - loss: 0.3548 - accuracy: 0.8497 - val_loss: 0.4017 - val_accuracy: 0.8247
Epoch 7/80
109/109 [==============================] - 3s 23ms/step - loss: 0.3027 - accuracy: 0.8867 - val_loss: 0.2672 - val_accuracy: 0.8866

In [12]:
model.evaluate(X_test,y_test)

8/8 [==============================] - 0s 18ms/step - loss: 0.0755 - accuracy: 0.9834


[0.07552309334278107, 0.9834024906158447]

In [15]:
from tensorflow.keras.models import save_model

# Save the trained model as an H5 file
model.save("smile_classifier_cnn.h5")

In [14]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
